In [14]:
train_val_data_ratio = 0.2
number_of_folds = 3
experiment = 'bugfix_big_4x'
synthetic_data = True
repeat_data_number = 4

# 1,3 - big
# 2   - clean
# 4   - messy
# 5   - synthetic 4x big

#TODO: #synthetically generate errors which we KNOW are due to incorrect line segmentation
#TODO: # Create dataset..

# 6   - synthetic 4x big
# 7   - synthetic 4x clean
# 8   - synthetic 4x messy


In [15]:
import csv
from aksharamukha import transliterate
import os
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
import random
import ast



csv_file = f'/home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_{experiment}/combined.csv'
directory = f'/home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_{experiment}'

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"Directory created: {directory}")
else:
    print(f"Directory already exists: {directory}")

Directory created: /home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_bugfix_big_4x


In [16]:
folder_name = 'big'

# Directory containing the files
directory = f'/home/ocr_proj/OCR/annotations/{folder_name}/text'
image_directory = f'/home/ocr_proj/OCR/annotations/{folder_name}/images'

# Initialize an empty string to store the merged content
merged_content_annot= []
merged_content_pred = []
image_paths = []

# Iterate over the range of file numbers
for i in range(12,52): # uncomment for big
#for i in range(2,43): # uncomment for clean
#for i in range(2, 17): # uncomment for messy
    # Construct the filename
    #if i not in [24, 29, 30, 31, 32, 33, 34, 35, 36, 37]: # uncomment for clean
        filename_annot = f'pg{i:04d}_annotated.txt'
        filepath_annot = os.path.join(directory, filename_annot)

        filename_pred = f'pg{i:04d}.txt'
        filepath_pred = os.path.join(directory, filename_pred)

        img = f'pg{i:04d}'
        img_path = os.path.join(image_directory, img)
        # Check if img_path exists and is a directory
        if os.path.exists(img_path) and os.path.isdir(img_path):
            # List all files in img_path directory
            files = os.listdir(img_path)
            files.sort()
            
            # Iterate through all files
            for file in files:
                # Check if the file is an image (you can adjust this condition as per your specific image extensions)
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    # Construct the full path of the image file
                    image_file_path = os.path.join(img_path, file)
                    # Add the image file path to the list
                    image_paths.append(image_file_path)
        else:
            print(f"Directory '{img_path}' does not exist.")
        
        # Open each file in read mode
        with open(filepath_annot, 'r') as infile:
            # Read the content and append it to the merged_content variable
            merged_content_annot.extend(infile.read().replace('||','॥').replace('।।','॥').replace('|','।').splitlines())

        # Open each file in read mode
        with open(filepath_pred, 'r') as infile:
            # Read the content and append it to the merged_content variable
            merged_content_pred.extend(infile.read().replace('||','॥').replace('।।','॥').replace('|','।').splitlines())
# merged_content now contains the merged text of all files

merged_content_annot = [transliterate.process('Devanagari', 'IAST', i) for i in merged_content_annot if i!='']
merged_content_pred = [transliterate.process('Devanagari', 'IAST', i) for i in merged_content_pred if i!='']
assert len(merged_content_annot) == len(merged_content_pred) == len(image_paths)
print("All files have been merged into the variable.")

data = zip(merged_content_pred, merged_content_annot, image_paths)
# Repeat each row 4 times

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    # Write header row
    writer.writerow(['input_text', 'target_text', 'path'])
    # Write data rows
    writer.writerows(data)


All files have been merged into the variable.


## Synthetic Errors

In [17]:
mistakes = {
"('delete', 'ं')": 102,
"('insert', 'ु')": 70,
"('insert', 'ं')": 61,
"('insert', 'ृ')": 52,
"('insert', 'े')": 43,
"('insert', 'ू')": 41,
"('delete', 'े')": 30,
"('replace', 'ो', 'ा')": 22
}

In [18]:
def process_and_normalize_dict(original_dict):
    # First, create a new dictionary with swapped action types and transliterated text
    processed_dict = {}
    
    for key, value in original_dict.items():
        # Extracting the tuple part from the string
        parts = key.strip("()").split(", ")
        action = parts[0].strip("'")
        text = parts[1].strip("'")
        if action == "replace":
            text2 = parts[2].strip("'")
            transliterated_text = transliterate.process('Devanagari', 'IAST', text)
            transliterated_text2 = transliterate.process('Devanagari', 'IAST', text2)
            
            new_key = f"('replace', '{transliterated_text2}', '{transliterated_text}')"
        else:
            # Transliterate the text
            transliterated_text = transliterate.process('Devanagari', 'IAST', text)
            
            # Swap action types
            new_action = 'delete' if action == 'insert' else 'insert'
            
            # Create new key with swapped action and transliterated text
            new_key = f"('{new_action}', '{transliterated_text}')"
        
        # Add the new key-value pair to the new dictionary
        processed_dict[new_key] = value
    
    # Calculate the total sum of values
    total_value = sum(processed_dict.values())
    
    # Normalize the values so that they sum up to one
    normalized_dict = {key: value / total_value for key, value in processed_dict.items()}
    
    return normalized_dict

# Perform the transliteration
new_dict = process_and_normalize_dict(mistakes)

In [19]:
combining_marks = ['्', 'ँ', 'ं', 'ः', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॢ', 'ॣ', 'े', 'ै', 'ो', 'ौ', '़','ॅ']

In [20]:
non_dependent_characters = [transliterate.process('Devanagari', 'IAST', text) for text in combining_marks]

In [21]:

def add_synthetic_errors(data_csv,error_dict):

    
    for index, row in data_csv.iterrows():
        # Access the first column value
        line = row[0]  # or row['ColumnName'] if using named columns
        line = line.strip()
        for key, prob in error_dict.items():

            key = ast.literal_eval(key)
            rule_type = key[0]
            
            c_1 = key[1]
            
            if rule_type == "delete":
                rand_delete = (
                    lambda c: "" if random.random() < prob and c == c_1 else c
                )
                line = "".join([rand_delete(c) for c in line])

            elif rule_type == "replace":
                        c_2 = key[2]
                        rand_replace = (
                            lambda c: c_2 if random.random() < prob and c == c_1 else c
                        )
                        line = "".join([rand_replace(c) for c in line])
                        
            elif rule_type == "insert":
                line = line + " "
                rand_insert = (
                    lambda c: "{}{}".format(c_1, c)
                    if random.random() < prob
                    and c not in non_dependent_characters else c
                )
                line = "".join([rand_insert(c) for c in line])

        data_csv.at[index, data_csv.columns[0]] = line.strip()
    return data_csv

                        

In [22]:
new_dict #synthetically generate errors which we KNOW are due to incorrect line segmentation.

{"('insert', 'ṃ')": 0.24228028503562946,
 "('delete', 'u')": 0.166270783847981,
 "('delete', 'ṃ')": 0.14489311163895488,
 "('delete', 'ṛ')": 0.12351543942992874,
 "('delete', 'e')": 0.1021377672209026,
 "('delete', 'ū')": 0.09738717339667459,
 "('insert', 'e')": 0.07125890736342043,
 "('replace', 'ā', 'o')": 0.052256532066508314}

In [23]:
#creating cross validation dasplits


# Load the CSV file
#csv_file = f'/home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/{experiment}/combined.csv'  # Replace with your CSV file path
data = pd.read_csv(csv_file, delimiter=';',encoding='utf-8')

# Initialize KFold with 10 splits
kf = KFold(n_splits=number_of_folds, shuffle=True, random_state=42)


# Counter for fold iteration
fold_count = 1

# Iterate over each fold
for train_val_index, test_index in kf.split(data):
    # Split data into training + validation and test sets
    train_val_data = data.iloc[train_val_index]
    test_data = data.iloc[test_index]


    # # Further split training + validation data into training and validation sets
    train_data, val_data = train_test_split(train_val_data, test_size=train_val_data_ratio, random_state=42)

    if synthetic_data:
        _train_data = pd.concat([train_data] * repeat_data_number, ignore_index=True)
        train_data = add_synthetic_errors(_train_data, new_dict)
    # headers = ['input_text', 'target_text']
    # train_data.columns = headers
    # val_data.columns = headers
    # test_data.columns = headers

    # Save the splits into separate CSV files
    train_data.to_csv(f'/home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_{experiment}/train_fold_{fold_count}.csv', sep=';', index=False, encoding='utf-8')
    val_data.to_csv(f'/home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_{experiment}/val_fold_{fold_count}.csv', sep=';', index=False, encoding='utf-8')
    test_data.to_csv(f'/home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_{experiment}/test_fold_{fold_count}.csv', sep=';', index=False, encoding='utf-8')

    # Increment fold count
    fold_count += 1

print('Data has been split into folds with train-validation-test splits.')


/tmp/ipykernel_1989772/2847755772.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  line = row[0]  # or row['ColumnName'] if using named columns


Data has been split into folds with train-validation-test splits.


In [13]:
# this code converts the raw .txt files to .csv. Change this if you want to do SLP1 instead of IAST or something..

# filename = 'big'
# # Define file names
# file1 = f'/home/ocr_proj/OCR/post_correction/denoising/data/{filename}_src.txt'
# file2 = f'/home/ocr_proj/OCR/post_correction/denoising/data/{filename}_tgt.txt'
# output_csv = '/home/ocr_proj/OCR/post_correction/denoising/data/combined.csv'

# # Open the two text files and the output CSV file
# with open(file1, 'r',encoding='utf-8') as f1, open(file2, 'r', encoding='utf-8') as f2, open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
#     csv_writer = csv.writer(csvfile, delimiter=';')

#     # Read lines from both files
#     lines1 = f1.readlines()
#     lines2 = f2.readlines()

    

#     # Combine lines line by line
#     for line1, line2 in zip(lines1, lines2):

#         #Transliterate
#         line1 = transliterate.process('Devanagari', 'IAST', line1)
#         line2 = transliterate.process('Devanagari', 'IAST', line2)
        
#         # Write combined lines to CSV
#         csv_writer.writerow([line1.strip(), line2.strip()])

# print(f'Combined data written to {output_csv}')